In [6]:
import boto3

In [7]:
ddb_client = boto3.client("dynamodb")

In [4]:
table_name = "Movies"

In [5]:
attributes = [
    {"AttributeName": "Title", "AttributeType": "S"},
    {"AttributeName": "Rating", "AttributeType": "N"},
]

In [6]:
key_schema = [
    {"AttributeName": "Title", "KeyType": "HASH"},
    {"AttributeName": "Rating", "KeyType": "RANGE"},
]

In [7]:
provisioned_throughput = {"ReadCapacityUnits": 1, "WriteCapacityUnits": 1}

In [ ]:
response = ddb_client.create_table(
    TableName=table_name,
    AttributeDefinitions=attributes,
    KeySchema=key_schema,
    ProvisionedThroughput=provisioned_throughput,
)

In [9]:
response

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Rating',
    'AttributeType': 'N'},
   {'AttributeName': 'Title', 'AttributeType': 'S'}],
  'TableName': 'Movies',
  'KeySchema': [{'AttributeName': 'Title', 'KeyType': 'HASH'},
   {'AttributeName': 'Rating', 'KeyType': 'RANGE'}],
  'TableStatus': 'CREATING',
  'CreationDateTime': datetime.datetime(2025, 5, 19, 10, 12, 35, 693000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:601621467089:table/Movies',
  'TableId': '2682606e-5024-4ebf-ab3e-a186077de2c2',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'R17P2F9HKJI41061TU6VQE85KFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 14:12:35 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length'

In [14]:
ddb_client.list_tables()

{'TableNames': ['Movies', 'cloud-gear-instances', 'cloud-gear-users'],
 'ResponseMetadata': {'RequestId': '4PUH323UNLPL0HHC909VNI3GJVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 14:14:30 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '67',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4PUH323UNLPL0HHC909VNI3GJVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '317816819'},
  'RetryAttempts': 0}}

In [15]:
entry = {
    "Title": {"S": "The Matrix"},
    "Director": {"S": "Lana Wachowski"},
    "Year": {"N": "1999"},
    "Rating": {"N": "5"},
}

In [16]:
ddb_client.put_item(TableName="Movies", Item=entry)

{'ResponseMetadata': {'RequestId': '605VVRAHMHQR1P9PMF6BQDQF27VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 14:27:29 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '605VVRAHMHQR1P9PMF6BQDQF27VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [17]:
item_key = {"Title": {"S": "The Matrix"}, "Rating": {"N": "5"}}

In [18]:
response = ddb_client.get_item(TableName=table_name, Key=item_key)

In [19]:
response["Item"]

{'Title': {'S': 'The Matrix'},
 'Director': {'S': 'Lana Wachowski'},
 'Year': {'N': '1999'},
 'Rating': {'N': '5'}}

In [20]:
update = "SET Director = :r"

In [27]:
ddb_client.update_item(
    TableName="Movies",
    Key=item_key,
    UpdateExpression=update,
    ExpressionAttributeValues={":r": {"S": "Lana and Lilly Wachowski"}},
)

{'ResponseMetadata': {'RequestId': '5SB20Q8BGKFUVC29B6J3PEL1OBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 15:27:34 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5SB20Q8BGKFUVC29B6J3PEL1OBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [1]:
movies = [
    {
        "Title": "The Matrix",
        "Director": "Lana Wachowski",
        "Year": "1999",
        "Rating": "4.7",
    },
    {
        "Title": "The Matrix 2",
        "Director": "Lana Wachowski",
        "Year": "2003",
        "Rating": "4.6",
    },
    {
        "Title": "The Matrix 3",
        "Director": "Lana Wachowski",
        "Year": "2003",
        "Rating": "4.5",
    },
    {
        "Title": "Inception",
        "Director": "Christopher Nolan",
        "Year": "2010",
        "Rating": "4.6",
    },
    {
        "Title": "Saving Private Ryan",
        "Director": "Steven Spielberg",
        "Year": "1999",
        "Rating": "4.7",
    },
]

In [12]:
batch_request = []
for movie in movies:
    batch_request.append(
        {
            "PutRequest": {
                "Item": {
                    "Title": {"S": movie["Title"]},
                    "Rating": {"N": str(movie["Rating"])},
                    "Director": {"S": movie["Director"]},
                    "Year": {"N": str(movie["Year"])},
                }
            }
        }
    )

In [13]:
response = ddb_client.batch_write_item(RequestItems={"Movies": batch_request})

In [14]:
batch_request

[{'PutRequest': {'Item': {'Title': {'S': 'The Matrix'},
    'Rating': {'N': '4.7'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '1999'}}}},
 {'PutRequest': {'Item': {'Title': {'S': 'The Matrix 2'},
    'Rating': {'N': '4.6'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '2003'}}}},
 {'PutRequest': {'Item': {'Title': {'S': 'The Matrix 3'},
    'Rating': {'N': '4.5'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '2003'}}}},
 {'PutRequest': {'Item': {'Title': {'S': 'Inception'},
    'Rating': {'N': '4.6'},
    'Director': {'S': 'Christopher Nolan'},
    'Year': {'N': '2010'}}}},
 {'PutRequest': {'Item': {'Title': {'S': 'Saving Private Ryan'},
    'Rating': {'N': '4.7'},
    'Director': {'S': 'Steven Spielberg'},
    'Year': {'N': '1999'}}}}]

In [16]:
batch_request_2 = {"Keys": []}
for movie in movies:
    batch_request_2["Keys"].append(
        {
            "Title": {"S": movie["Title"]},
            "Rating": {"N": str(movie["Rating"])},
        }
    )

In [19]:
ddb_client.batch_get_item(RequestItems={"Movies": batch_request_2})

{'Responses': {'Movies': [{'Title': {'S': 'The Matrix'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '1999'},
    'Rating': {'N': '4.7'}},
   {'Title': {'S': 'The Matrix 2'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '2003'},
    'Rating': {'N': '4.6'}},
   {'Title': {'S': 'The Matrix 3'},
    'Director': {'S': 'Lana Wachowski'},
    'Year': {'N': '2003'},
    'Rating': {'N': '4.5'}},
   {'Title': {'S': 'Inception'},
    'Director': {'S': 'Christopher Nolan'},
    'Year': {'N': '2010'},
    'Rating': {'N': '4.6'}},
   {'Title': {'S': 'Saving Private Ryan'},
    'Director': {'S': 'Steven Spielberg'},
    'Year': {'N': '1999'},
    'Rating': {'N': '4.7'}}]},
 'UnprocessedKeys': {},
 'ResponseMetadata': {'RequestId': '7O6KT60DJE6TRO3T591UN960NFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 18:42:41 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '584',
   '

In [17]:
ddb_client.list_tables()

{'TableNames': ['Movies', 'cloud-gear-instances', 'cloud-gear-users'],
 'ResponseMetadata': {'RequestId': 'KBRQ30LPLK8UCUC2MID2SBRAGNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 18:42:02 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '67',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'KBRQ30LPLK8UCUC2MID2SBRAGNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '317816819'},
  'RetryAttempts': 0}}

In [21]:
items = []
response = ddb_client.scan(TableName="Movies")
items.extend(response["Items"])

while "LastEvaluatedKey" in response.keys():
    response = ddb_client.scan(
        TableName="Movies", ExclusiveStartKey=response["LastEvaluatedKey"]
    )
    items.extend(response["Items"])

In [22]:
items

[{'Title': {'S': 'The Matrix'},
  'Director': {'S': 'Lana Wachowski'},
  'Year': {'N': '1999'},
  'Rating': {'N': '4.7'}},
 {'Title': {'S': 'The Matrix'},
  'Director': {'S': 'Lana and Lilly Wachowski'},
  'Year': {'N': '1999'},
  'Rating': {'N': '5'}},
 {'Title': {'S': 'Inception'},
  'Director': {'S': 'Christopher Nolan'},
  'Year': {'N': '2010'},
  'Rating': {'N': '4.6'}},
 {'Title': {'S': 'The Matrix 3'},
  'Director': {'S': 'Lana Wachowski'},
  'Year': {'N': '2003'},
  'Rating': {'N': '4.5'}},
 {'Title': {'S': 'Saving Private Ryan'},
  'Director': {'S': 'Steven Spielberg'},
  'Year': {'N': '1999'},
  'Rating': {'N': '4.7'}},
 {'Title': {'S': 'The Matrix 2'},
  'Director': {'S': 'Lana Wachowski'},
  'Year': {'N': '2003'},
  'Rating': {'N': '4.6'}}]

In [24]:
ddb_client.scan(
    TableName="Movies",
    FilterExpression="Rating >= :num",
    ExpressionAttributeValues={":num": {"N": "4.7"}},
)

{'Items': [{'Title': {'S': 'The Matrix'},
   'Director': {'S': 'Lana Wachowski'},
   'Year': {'N': '1999'},
   'Rating': {'N': '4.7'}},
  {'Title': {'S': 'The Matrix'},
   'Director': {'S': 'Lana and Lilly Wachowski'},
   'Year': {'N': '1999'},
   'Rating': {'N': '5'}},
  {'Title': {'S': 'Saving Private Ryan'},
   'Director': {'S': 'Steven Spielberg'},
   'Year': {'N': '1999'},
   'Rating': {'N': '4.7'}}],
 'Count': 3,
 'ScannedCount': 6,
 'ResponseMetadata': {'RequestId': '3VTDI7B5L8FBTCPP4U23GLGFBJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 19 May 2025 19:29:57 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '369',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3VTDI7B5L8FBTCPP4U23GLGFBJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3134383722'},
  'RetryAttempts': 0}}